In [2]:
import os
import sys


os.chdir("/teamspace/studios/this_studio/nurse-helper-chatbot")


sys.path.append(os.getcwd())

In [35]:
import streamlit as st
from llm import llm, embeddings
from graph import graph
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate


neo4jvector = Neo4jVector.from_existing_index(
    embedding=embeddings,
    index_name="reviewEmbeddingIndex",
    node_label="review",
    text_node_property="Review",
    embedding_node_property="embedding",   
)


retriever = neo4jvector.as_retriever(k=12)



instructions = (
    "Your job is to use patient reviews to answer questions about their experience at a hospital. Use the following context to answer questions. Be as detailed as possible, but don't make up any information that's not from the context. If you don't know an answer, say you don't know."
    "If you don't know the answer, say you don't know."
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", instructions),
        ("human", "{input}"),
    ]
)



question_answer_chain = create_stuff_documents_chain(llm, prompt)
review_retriever = create_retrieval_chain(
    retriever, 
    question_answer_chain
)



def get_review(input):
    return review_retriever.invoke({"input": input})


In [34]:
# Example input question
input_question = "How's the review of foods or cafeteria of hospitals?"

# Get the review result
result = get_review(input_question)

# Display the result
print("Review Result:", result)

Review Result: {'input': "How's the review of foods or cafeteria of hospitals?", 'context': [Document(metadata={'Hospital_ID': 8, 'Patient_ID': 162, 'Checkup_ID': 259, 'Review_ID': 186}, page_content='I received outstanding care from the hospital staff. The facilities were state-of-the-art, but the cafeteria options were limited.'), Document(metadata={'Hospital_ID': 13, 'Patient_ID': 366, 'Checkup_ID': 254, 'Review_ID': 127}, page_content='The medical staff was efficient, and the hospital rooms were clean and comfortable. On the downside, the cafeteria food was bland, and the menu was repetitive.'), Document(metadata={'Hospital_ID': 9, 'Patient_ID': 423, 'Checkup_ID': 298, 'Review_ID': 115}, page_content='I had a positive experience overall at the hospital. The medical staff was skilled, and the facilities were modern. The only downside was the limited menu in the cafeteria.'), Document(metadata={'Hospital_ID': 6, 'Patient_ID': 613, 'Checkup_ID': 504, 'Review_ID': 230}, page_content='T

Creating Index pls dont run it

In [28]:
db = Neo4jVector.from_existing_graph(
    embedding=embeddings,
    index_name="reviewEmbeddingIndex",
    node_label="review",
    text_node_properties=["Review"],
    embedding_node_property="embedding",
)

In [27]:
# Example input question
input_question = "How's the review of foods or cafeteria of hospitals?"

# Get the review result
result = get_review(input_question)

# Display the result
print("Review Result:", result)

Review Result: {'input': "How's the review of foods or cafeteria of hospitals?", 'context': [], 'answer': "I don't know."}
